# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from absa_models.ABSA_BERT_Dropout_Linear import ABSA_BERT_Dropout_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
BERT_FINE_TUNED_PATH = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [11]:
MODEL_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_linear.pth'
STATS_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_linear.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_Linear(torch.load(BERT_FINE_TUNED_PATH), dropout=0.3, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.2192856073379517
Epoch: 0/2, Batch: 50/501, Loss: 0.14524944126605988
Epoch: 0/2, Batch: 100/501, Loss: 0.07134665548801422
Epoch: 0/2, Batch: 150/501, Loss: 0.022834982722997665
Epoch: 0/2, Batch: 200/501, Loss: 0.03208481892943382
Epoch: 0/2, Batch: 250/501, Loss: 0.11824347078800201
Epoch: 0/2, Batch: 300/501, Loss: 0.0375695563852787
Epoch: 0/2, Batch: 350/501, Loss: 0.06119386479258537
Epoch: 0/2, Batch: 400/501, Loss: 0.0995965525507927
Epoch: 0/2, Batch: 450/501, Loss: 0.037751272320747375
Epoch: 0/2, Batch: 500/501, Loss: 0.034646376967430115
Epoch: 1/2, Batch: 0/501, Loss: 0.027884313836693764
Epoch: 1/2, Batch: 50/501, Loss: 0.017969103530049324
Epoch: 1/2, Batch: 100/501, Loss: 0.0761646255850792
Epoch: 1/2, Batch: 150/501, Loss: 0.0815887600183487
Epoch: 1/2, Batch: 200/501, Loss: 0.01859542541205883
Epoch: 1/2, Batch: 250/501, Loss: 0.044734127819538116
Epoch: 1/2, Batch: 300/501, Loss: 0.04084711894392967
Epoch: 1/2, Batch: 350/5

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.349490761756897
Epoch: 0/2, Batch: 50/501, Loss: 0.07041408866643906
Epoch: 0/2, Batch: 100/501, Loss: 0.03137765824794769
Epoch: 0/2, Batch: 150/501, Loss: 0.1312115341424942
Epoch: 0/2, Batch: 200/501, Loss: 0.0369698703289032
Epoch: 0/2, Batch: 250/501, Loss: 0.04585210606455803
Epoch: 0/2, Batch: 300/501, Loss: 0.014551891945302486
Epoch: 0/2, Batch: 350/501, Loss: 0.015732765197753906
Epoch: 0/2, Batch: 400/501, Loss: 0.0055825901217758656
Epoch: 0/2, Batch: 450/501, Loss: 0.07993457466363907
Epoch: 0/2, Batch: 500/501, Loss: 0.07829033583402634
Epoch: 1/2, Batch: 0/501, Loss: 0.014512639492750168
Epoch: 1/2, Batch: 50/501, Loss: 0.06725473701953888
Epoch: 1/2, Batch: 100/501, Loss: 0.06459648907184601
Epoch: 1/2, Batch: 150/501, Loss: 0.030837060883641243
Epoch: 1/2, Batch: 200/501, Loss: 0.05601300299167633
Epoch: 1/2, Batch: 250/501, Loss: 0.04259899631142616
Epoch: 1/2, Batch: 300/501, Loss: 0.01589256525039673
Epoch: 1/2, Batch: 350/

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 4/10
Epoch: 0/2, Batch: 0/501, Loss: 1.371902346611023
Epoch: 0/2, Batch: 50/501, Loss: 0.07042928040027618
Epoch: 0/2, Batch: 100/501, Loss: 0.09496767073869705
Epoch: 0/2, Batch: 150/501, Loss: 0.08275280147790909
Epoch: 0/2, Batch: 200/501, Loss: 0.08349484205245972
Epoch: 0/2, Batch: 250/501, Loss: 0.011116023175418377
Epoch: 0/2, Batch: 300/501, Loss: 0.0663209855556488
Epoch: 0/2, Batch: 350/501, Loss: 0.04129355400800705
Epoch: 0/2, Batch: 400/501, Loss: 0.05127806216478348
Epoch: 0/2, Batch: 450/501, Loss: 0.1577591598033905
Epoch: 0/2, Batch: 500/501, Loss: 0.042101629078388214
Epoch: 1/2, Batch: 0/501, Loss: 0.04772809520363808
Epoch: 1/2, Batch: 50/501, Loss: 0.031473610550165176
Epoch: 1/2, Batch: 100/501, Loss: 0.07644002884626389
Epoch: 1/2, Batch: 150/501, Loss: 0.04348011314868927
Epoch: 1/2, Batch: 200/501, Loss: 0.04835059866309166
Epoch: 1/2, Batch: 250/501, Loss: 0.04368799924850464
Epoch: 1/2, Batch: 300/501, Loss: 0.02715257555246353
Epoch: 1/2, Batch: 350/501

...

Run 5/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3295319080352783
Epoch: 0/2, Batch: 50/501, Loss: 0.03148853778839111
Epoch: 0/2, Batch: 100/501, Loss: 0.09085922688245773
Epoch: 0/2, Batch: 150/501, Loss: 0.07800865918397903
Epoch: 0/2, Batch: 200/501, Loss: 0.042824629694223404
Epoch: 0/2, Batch: 250/501, Loss: 0.048986587673425674
Epoch: 0/2, Batch: 300/501, Loss: 0.04246745631098747
Epoch: 0/2, Batch: 350/501, Loss: 0.11031121760606766
Epoch: 0/2, Batch: 400/501, Loss: 0.10655477643013
Epoch: 0/2, Batch: 450/501, Loss: 0.04407835006713867
Epoch: 0/2, Batch: 500/501, Loss: 0.021373648196458817
Epoch: 1/2, Batch: 0/501, Loss: 0.029239878058433533
Epoch: 1/2, Batch: 50/501, Loss: 0.03691225126385689
Epoch: 1/2, Batch: 100/501, Loss: 0.012606032192707062
Epoch: 1/2, Batch: 150/501, Loss: 0.024339891970157623
Epoch: 1/2, Batch: 200/501, Loss: 0.10054315626621246
Epoch: 1/2, Batch: 250/501, Loss: 0.008461270481348038
Epoch: 1/2, Batch: 300/501, Loss: 0.02229062281548977
Epoch: 1/2, Batch: 350

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 6/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3917652368545532
Epoch: 0/2, Batch: 50/501, Loss: 0.14666056632995605
Epoch: 0/2, Batch: 100/501, Loss: 0.3779398202896118
Epoch: 0/2, Batch: 150/501, Loss: 0.09269756078720093
Epoch: 0/2, Batch: 200/501, Loss: 0.0840015858411789
Epoch: 0/2, Batch: 250/501, Loss: 0.2264525294303894
Epoch: 0/2, Batch: 300/501, Loss: 0.07303115725517273
Epoch: 0/2, Batch: 350/501, Loss: 0.02259303815662861
Epoch: 0/2, Batch: 400/501, Loss: 0.03601498529314995
Epoch: 0/2, Batch: 450/501, Loss: 0.04114999994635582
Epoch: 0/2, Batch: 500/501, Loss: 0.06529584527015686
Epoch: 1/2, Batch: 0/501, Loss: 0.024579845368862152
Epoch: 1/2, Batch: 50/501, Loss: 0.07129933685064316
Epoch: 1/2, Batch: 100/501, Loss: 0.05062153935432434
Epoch: 1/2, Batch: 150/501, Loss: 0.025184225291013718
Epoch: 1/2, Batch: 200/501, Loss: 0.027115756645798683
Epoch: 1/2, Batch: 250/501, Loss: 0.08201992511749268
Epoch: 1/2, Batch: 300/501, Loss: 0.007353530265390873
Epoch: 1/2, Batch: 350/50

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 8/10
Epoch: 0/2, Batch: 0/501, Loss: 1.529990792274475
Epoch: 0/2, Batch: 50/501, Loss: 0.04068974778056145
Epoch: 0/2, Batch: 100/501, Loss: 0.1398448497056961
Epoch: 0/2, Batch: 150/501, Loss: 0.17523221671581268
Epoch: 0/2, Batch: 200/501, Loss: 0.02684064954519272
Epoch: 0/2, Batch: 250/501, Loss: 0.010592305101454258
Epoch: 0/2, Batch: 300/501, Loss: 0.11210278421640396
Epoch: 0/2, Batch: 350/501, Loss: 0.018953820690512657
Epoch: 0/2, Batch: 400/501, Loss: 0.025227978825569153
Epoch: 0/2, Batch: 450/501, Loss: 0.03870471194386482
Epoch: 0/2, Batch: 500/501, Loss: 0.13883239030838013
Epoch: 1/2, Batch: 0/501, Loss: 0.03762127831578255
Epoch: 1/2, Batch: 50/501, Loss: 0.017697904258966446
Epoch: 1/2, Batch: 100/501, Loss: 0.04259184002876282
Epoch: 1/2, Batch: 150/501, Loss: 0.009015156887471676
Epoch: 1/2, Batch: 200/501, Loss: 0.014062333852052689
Epoch: 1/2, Batch: 250/501, Loss: 0.10408025234937668
Epoch: 1/2, Batch: 300/501, Loss: 0.05302978679537773
Epoch: 1/2, Batch: 350

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.983592,0.983592,0.914755,0.983592,0.719106,0.983592,0.719788,111.428999
1,0.982829,0.982829,0.665687,0.982829,0.686409,0.982829,0.674743,105.950998
2,0.988978,0.988978,0.688113,0.988978,0.713356,0.988978,0.700026,112.231005
3,0.986627,0.986627,0.663254,0.986627,0.714438,0.986627,0.684872,110.252000
4,0.987628,0.987628,0.687074,0.987628,0.712026,0.987628,0.699044,109.548000
5,0.985831,0.985831,0.920792,0.985831,0.744869,0.985831,0.752467,111.006002
6,0.985644,0.985644,0.677406,0.985644,0.707978,0.985644,0.691803,111.963002
7,0.984780,0.984780,0.925881,0.984780,0.698788,0.984780,0.717338,109.810998
8,0.982809,0.982809,0.933443,0.982809,0.692413,0.982809,0.721876,109.367999
9,0.984983,0.984983,0.937207,0.984983,0.729587,0.984983,0.757635,110.550999


In [20]:
results.to_csv(STATS_OUTPUT)